# Analyzing CIA Factbook Data Using SQL

Aim of this project is to explore & analyze demographic information. The data comes from [CIA World Factbook](https://www.cia.gov/library/publications/the-world-factbook/), a compendium of statistics about all of the countries on Earth The Factbook contains demographic information like:

- population - The population as of 2015.
- population_growth - The annual population growth rate, as a percentage.
- area - The total land and water area.


In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

Let's explore first 5 rows of the table.

In [3]:
%%sql
SELECT *
    FROM facts
  LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are the description for some of the columns:

- name - The name of the country.
- area- The country's total area (both land and water).
- area_land - The country's land area in square kilometers.
- area_water - The country's waterarea in square kilometers.
- population - The country's population.
- population_growth- The country's population growth as a percentage.
- birth_rate - The country's birth rate, or the number of births a year per 1,000 people.
- death_rate - The country's death rate, or the number of death a year per 1,000 people.

### Summary statistics

In [6]:
%%sql
SELECT MIN(population) AS min_pop, 
        MAX(population) AS max_pop, 
        MIN(population_growth) AS min_pop_growth,
        MAX(population_growth) AS max_pop_growth
    FROM facts;

Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


From the table above we can conclude that:

- some country has 0 population what we have to explore further,
- max_population of a country is 7,256,490,011 that also required further analysis. we explore that there are outlier countries

First let's explore which country has a population equal 0.

In [10]:
%%sql
SELECT *
    FROM facts
    WHERE population == (SELECT MIN(population)
                         FROM facts)

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


From the table above we can see that population is 0 for Antarctica. Which makes sense and match the CIA Factbook [page for Antarctica](https://www.cia.gov/library/publications/the-world-factbook/geos/ay.html).

![title](Antarctica.png)

Next, let's see the row with maximum population.

In [11]:
%%sql
SELECT *
    FROM facts
    WHERE population == (SELECT MAX(population)
                        FROM facts)

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


As we can see from the table above the row that contains max population refers to the entire world, which explains the maximum population of over 7.2 billion we found earlier.

Now we can recalculate the summary statistics calculated earlier, excluding the row for the whole world.

In [16]:
%%sql
SELECT MIN(population) AS min_pop, 
        MAX(population) AS max_pop, 
        MIN(population_growth) AS min_pop_growth,
        MAX(population_growth) AS max_pop_growth
    FROM facts
    WHERE population != (SELECT MAX(population)
                        FROM facts);

Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,1367485388,0.0,4.02


Max_population is 1,367,485,388 which is logic value , let's discover it's country.

In [47]:
%%sql
SELECT * 
    FROM facts
    WHERE population=(
        SELECT MAX(population) FROM facts
        WHERE name != 'World');

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44


China have the maximum population 1,367,485,388 with population growth rate 0.45

### Population and Area

In [48]:
%%sql
-- calculate the average value for both population & area
SELECT AVG(population) AS avg_population, AVG(area) AS avg_area
    FROM facts
    WHERE name <> 'World';

Done.


avg_population,avg_area
32242666.56846473,555093.546184739


From our output the average population is around 32 million whereas the average area is approximately 555 thousand Kilometer square

Now let us explore which countries are densely populated by find:

- Countries above average values for population
- and Below average values for area

In [58]:
%%sql
SELECT *
    FROM facts
  WHERE population > (SELECT AVG(population)
                           FROM facts
                          WHERE name <> 'World'
                       )
  AND area < (SELECT AVG(area)
                     FROM facts
                    WHERE name <> 'World'
                 )

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


In conclusion, if we compare our result with the real data of some of these countries, we will find the result to be true.

### People

In [51]:
%%sql
-- Country has the most people? 
SELECT * 
    FROM facts
    WHERE population =(SELECT MAX(population) FROM facts
                      WHERE name <> 'World');

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44


As found before China have the maximum population 1,367,485,388 with population growth rate 0.45. Let's see which country has the highest growth rate.

In [27]:
%%sql 
SELECT *
    FROM facts
 WHERE population_growth == (SELECT MAX(population_growth)
                                       FROM facts
                                       WHERE name <> 'World');

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
162,od,South Sudan,644329,None,None,12042910,4.02,36.91,8.18,11.47


South Sudan has the highest growth rate 4.02.

In [59]:
%%sql
SELECT *
    FROM facts
    WHERE migration_rate = (SELECT MAX(migration_rate) FROM facts
                            WHERE name <> 'World')

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
141,qa,Qatar,11586,11586,0,2194817,3.07,9.84,1.53,22.39


Qatar from our output has the highest migration rate

In [60]:
%%sql
SELECT *
    FROM facts
WHERE population * population_growth/100 = (SELECT MAX(population * population_growth/100)
                        FROM facts
                       WHERE name <> 'World');

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
77,in,India,3287263,2973193,314070,1251695584,1.22,19.55,7.32,0.04


India has the highest probablity of adding to their population next year based on the ratio between their population and population growth.

### Water to Land Ratios

In [53]:
%%sql
SELECT name, CAST(area_water AS FLOAT)/CAST(area AS FLOAT) AS ratio_water_to_land
    FROM facts
    ORDER BY ratio_water_to_land DESC;

Done.


name,ratio_water_to_land
British Indian Ocean Territory,0.9988970588235294
Virgin Islands,0.818848167539267
Puerto Rico,0.35682691610470596
"Bahamas, The",0.2788184438040346
Guinea-Bissau,0.2215916955017301
Malawi,0.20596873839505755
Netherlands,0.1841465469513516
Uganda,0.18228661040997685
Eritrea,0.141156462585034
Liberia,0.13512736937567904


British Indian Ocean Territory has the highest water ratio with percentage about 99.9%
Virgin Islands has water area than land area with percentage about 81.9%.

In [37]:
%%sql
SELECT *
    FROM facts
    WHERE area_water > area_land;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
228,io,British Indian Ocean Territory,54400,60,54340,None,None,None,None,None
247,vq,Virgin Islands,1910,346,1564,103574,0.59,10.31,8.54,7.67


### Birth rate VS Death rate

Let's look at the table again.

In [38]:
%%sql
SELECT *
    FROM facts;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46
6,ac,Antigua and Barbuda,442,442,0,92436,1.24,15.85,5.69,2.21
7,ar,Argentina,2780400,2736690,43710,43431886,0.93,16.64,7.33,0.0
8,am,Armenia,29743,28203,1540,3056382,0.15,13.61,9.34,5.8
9,as,Australia,7741220,7682300,58920,22751014,1.07,12.15,7.14,5.65
10,au,Austria,83871,82445,1426,8665550,0.55,9.41,9.42,5.56


In [54]:
%%sql 
SELECT * 
    FROM facts
    WHERE birth_rate == (SELECT MAX(birth_rate)
                            FROM facts);

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
128,ng,Niger,None,1266700,300,18045729,3.25,45.45,12.42,0.56


Niger has the maximum birth_rate 45.45 it's population 18045729 sharing area 1266700.

In [61]:
%%sql
SELECT *
    FROM facts
    WHERE death_rate > birth_rate AND name <> 'World'
    ORDER BY death_rate DESC;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
183,up,Ukraine,603550,579330,24220,44429471,0.6,10.72,14.46,2.25
26,bu,Bulgaria,110879,108489,2390,7186893,0.58,8.92,14.44,0.29
96,lg,Latvia,64589,62249,2340,1986705,1.06,10.0,14.31,6.26
102,lh,Lithuania,65300,62680,2620,2884433,1.04,10.1,14.27,6.27
143,rs,Russia,17098242,16377742,720500,142423773,0.04,11.6,13.69,1.69
153,ri,Serbia,77474,77474,0,7176794,0.46,9.08,13.66,0.0
16,bo,Belarus,207600,202900,4700,9589689,0.2,10.7,13.36,0.7
75,hu,Hungary,93028,89608,3420,9897541,0.22,9.16,12.73,1.33
116,md,Moldova,33851,32891,960,3546847,1.03,12.0,12.59,9.67
57,en,Estonia,45228,42388,2840,1265420,0.55,10.51,12.4,3.6


Ukraine has the highest death_rate around 14.46.

### Area/Population Ratio

In [57]:
%%sql
SELECT name, CAST(population AS FLOAT)/CAST(area AS FLOAT) AS ratio_pop_area
    FROM facts
    
    ORDER BY ratio_pop_area DESC;

Done.


name,ratio_pop_area
Macau,21168.964285714286
Monaco,15267.5
Singapore,8141.279770444763
Hong Kong,6445.041516245487
Gaza Strip,5191.819444444444
Gibraltar,4876.333333333333
Bahrain,1771.8592105263158
Maldives,1319.6409395973155
Malta,1310.01582278481
Bermuda,1299.9259259259259


Macau has the highest population/area ratio and the area of land in square kilometer is so small posibly compared to other countries.